# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from tqdm import tqdm
from time import time

In [2]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

#X_val = np.load('data/x_val.npy')
#y_val = np.load('data/y_val.npy')

In [3]:
#X_train = np.concatenate((X_train, X_val), axis=0)
#y_train = np.concatenate((y_train, y_val), axis=0)

# Playground

In [4]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3, 4, 5], dtype=int8),
 array([ 10150,  91156,    429, 115824,   7646,   6441]))

3 is normal class

In [5]:
normal_data = X_train[y_train == 3] # normal class

# Helpers

In [6]:
def evaluate_model(model, y_test, y_pred):
    positive_class = 'normal'
    
    accuracy = accuracy_score(y_test, y_pred)
    
    precision = precision_score(y_test, y_pred, pos_label=positive_class)
    recall = recall_score(y_test, y_pred, pos_label=positive_class)
    f1 = f1_score(y_test, y_pred, pos_label=positive_class)
    
    metrics = {
        'model': [model],
        'accuracy': [accuracy],
        'precision': [precision],
        'recall': [recall],
        'f1': [f1],
    }
    
    return metrics

# SGD-OCSVM

In [7]:
from sklearn.linear_model import SGDOneClassSVM

In [55]:
sgd_ocsvm_start = time()

sgd_ocsvm = SGDOneClassSVM(nu=0.05, max_iter=1000, tol=0.001, learning_rate='adaptive', eta0=0.05, random_state=42)
sgd_ocsvm.fit(normal_data)
        
print(f"seconds: {time() - sgd_ocsvm_start}")

seconds: 0.7971386909484863


In [56]:
y_pred_test_sgd_ocsvm = sgd_ocsvm.predict(X_test)
y_pred_test_sgd_ocsvm = np.where(y_pred_test_sgd_ocsvm == 1, 'normal', 'anomaly')
y_test_converted_sgd_ocsvm = np.where(y_test == 3, 'normal', 'anomaly')

In [57]:
sgd_ocsvm_df = pd.DataFrame(evaluate_model("sgd_ocsvm", y_test_converted_sgd_ocsvm, y_pred_test_sgd_ocsvm))
sgd_ocsvm_df

,model,accuracy,precision,recall,f1
0,sgd_ocsvm,0.856864,0.778326,0.997905,0.874543


# PCA Reconstruction

In [12]:
from sklearn.decomposition import PCA

In [76]:
pca_start = time()

pca = PCA(n_components=3, whiten=True, svd_solver='auto')

normal_data_pca = pca.fit_transform(normal_data)
normal_data_reconstructed = pca.inverse_transform(normal_data_pca)

reconstruction_error = np.mean((normal_data - normal_data_reconstructed) ** 2, axis=1)

threshold = np.percentile(reconstruction_error, 95)

X_test_pca = pca.transform(X_test)
X_test_reconstructed = pca.inverse_transform(X_test_pca)
X_test_reconstruction_error = np.mean((X_test - X_test_reconstructed) ** 2, axis=1)

print(f"seconds: {time() - pca_start}")

seconds: 1.5816032886505127


In [77]:
y_pred_test_pca = np.where(X_test_reconstruction_error > threshold, 'anomaly', 'normal')
y_test_converted_pca = np.where(y_test == 3, 'normal', 'anomaly')

In [78]:
pca_df = pd.DataFrame(evaluate_model('PCA Reconstruction', y_test_converted_pca, y_pred_test_pca))
pca_df

,model,accuracy,precision,recall,f1
0,PCA Reconstruction,0.825582,0.744312,0.991841,0.850431


# Results (2)

In [79]:
results_df = pd.concat([sgd_ocsvm_df, pca_df], ignore_index=True)
results_df.to_csv('results/results_mqtt_2.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,sgd_ocsvm,0.856864,0.778326,0.997905,0.874543
1,PCA Reconstruction,0.825582,0.744312,0.991841,0.850431


# OCSVM

In [52]:
from sklearn.svm import OneClassSVM

In [87]:
ocsvm = OneClassSVM(kernel='rbf', gamma=0.05, nu=0.1)

batch_size = 50000
n_batches = 1# int(np.ceil(len(normal_data) / batch_size))

for i in tqdm(range(n_batches), desc="Training OCSVM"):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(normal_data))
    batch_data = normal_data[start:end]
    
    if i == 0:
        ocsvm.fit(batch_data)
    else:
        ocsvm.fit(np.vstack((ocsvm.support_vectors_, batch_data)))

Training OCSVM: 100%|██████████| 1/1 [00:24<00:00, 24.83s/it]


In [88]:
y_pred_test_ocsvm = ocsvm.predict(X_test)
y_pred_test_ocsvm = np.where(y_pred_test_ocsvm == 1, 'normal', 'anomaly')
y_test_converted_ocsvm = np.where(y_test == 3, 'normal', 'anomaly')

In [89]:
evaluate_model("ocsvm", y_test_converted_ocsvm, y_pred_test_ocsvm)

{'model': ['ocsvm'],
 'accuracy': [0.840678819619297],
 'precision': [0.7873406966864911],
 'recall': [0.9334394327041238],
 'f1': [0.854187982191743]}

In [83]:
ocsvm_df = pd.DataFrame(evaluate_model("ocsvm", y_test_converted_ocsvm, y_pred_test_ocsvm))
ocsvm_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.840679,0.787341,0.933439,0.854188


# LOF

In [90]:
from sklearn.neighbors import LocalOutlierFactor

In [115]:
lof_start = time()

lof = LocalOutlierFactor(n_neighbors=200, contamination=0.5, novelty=True, leaf_size=40)
lof.fit(X_train)

print(f"seconds: {time() - lof_start}")

seconds: 31.28235673904419


In [116]:
y_pred_test_lof = lof.predict(X_test)
y_pred_test_lof = np.where(y_pred_test_lof == 1, 'normal', 'anomaly')
y_test_converted_lof = np.where(y_test == 3, 'normal', 'anomaly')

In [117]:
lof_df = pd.DataFrame(evaluate_model("lof", y_test_converted_lof, y_pred_test_lof)) 
lof_df

,model,accuracy,precision,recall,f1
0,lof,0.538443,0.557351,0.373053,0.446949


# IF

In [118]:
from sklearn.ensemble import IsolationForest

In [155]:
if_start = time()

iforest = IsolationForest(n_estimators=100, max_samples=256, contamination=0.3, random_state=42)
iforest.fit(X_train)

print(f"seconds: {time() - if_start}")

seconds: 5.823337554931641


In [156]:
y_pred_test_if = iforest.predict(X_test)
y_pred_test_if = np.where(y_pred_test_if == 1, 'normal', 'anomaly')
y_test_converted_if = np.where(y_test == 3, 'normal', 'anomaly')

In [157]:
evaluate_model("if", y_test_converted_if, y_pred_test_if)

{'model': ['if'],
 'accuracy': [0.7519689797562695],
 'precision': [0.6732420900980772],
 'recall': [0.9790688772940631],
 'f1': [0.797852692752838]}

In [158]:
if_df = pd.DataFrame(evaluate_model("if", y_test_converted_if, y_pred_test_if)) 
if_df

,model,accuracy,precision,recall,f1
0,if,0.751969,0.673242,0.979069,0.797853


# Results

In [160]:
results_df = pd.concat([ocsvm_df, lof_df, if_df], ignore_index=True)
results_df.to_csv('results/results_mqtt.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.840679,0.787341,0.933439,0.854188
1,lof,0.538443,0.557351,0.373053,0.446949
2,if,0.751969,0.673242,0.979069,0.797853
